## import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.fftpack as fft
import poppy
import gpipsfs
import pytz
import time
import re
import os

from scipy import signal
from scipy.signal import welch
from astropy.io import fits
from scipy import optimize

## Telescope dimensions

In [ ]:
outD = 7.77010            # primary diameter (m)
inD = 1.024               # inner M2 diameter (m)
n = 48                    # number sample points across the screen (Not the number of subapertures)
nacross = 43              # number of subapertures across the aperture
pscale = outD/(nacross)   # pixel size (m) of samples in pupil plane

## Phase sample

In [ ]:
def path_manager(desktop_name):
    
    if desktop_name == 'gpi_cruncher':
        rootdir = '/home/sda/mtallis/PhaseScripts/aotelem/Reduced/'
        save_path = '/home/sda/mtallis/Results/datatables/'
        samples_path = '/home/sda/mtallis/samples/phase_samples_20181119.txt'
        #rootdir = '/home/sda/mtallis/PhaseScripts/aotelem/Reduced/'
        #save_path = '/home/sda/mtallis/Results/c_Eri/'
        #samples_path = '/home/sda/mtallis/samples/c_Eri_samples.txt'
    
    if desktop_name == 'kipac':
        rootdir = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        save_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        samples_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/datatables/'
        
    if desktop_name == 'laptop':
        rootdir = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        save_path = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        samples_path = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/c_Eri_samples.txt'

    dstr = time.strftime('%Y%m%d')
    return rootdir,save_path, samples_path, dstr

In [ ]:
rootdir, save_path, samples_path, dstr = path_manager('gpi_cruncher')

In [ ]:
with open(samples_path,'r') as f:
    sample = f.read().splitlines() #outputs as a list of strings

In [ ]:
fname_list = list()
name_list = list()

for i in sample:
    for root, dirs, files in os.walk(rootdir):
        for name in files:
            (base,ext) = os.path.splitext(name)
            if (ext in ('.fits')) and (i in base):
                full_name = os.path.join(root,name)
                #print(full_name)
                fname_list.append(full_name)  
                name_list.append(base[11:-12])

## Analysis Functions

In [ ]:
def process_phase(filepath):

    hdulist = fits.open(filepath,memmap=True)
    phase = hdulist[0].data.astype('float')
    avg_phase = np.nanmean(phase*ap_nan,axis=0)  # used to find average zernikes 

    # remove zernikes form cube
    z_basis = poppy.zernike.zernike_basis_faster(nterms= 6, npix = 48)
    z_coeff = poppy.zernike.opd_expand_nonorthonormal(avg_phase, aperture=ap, nterms=6)
    thin_lens = np.sum(z_coeff[:,None,None]*z_basis[:,:,:],axis=0)

    c_phase = (phase - thin_lens[None,:,:])*ap_nan
    c_phase[np.isnan(c_phase)]=0.
    
    return c_phase

def power_law_fit(k,Y,low_b,up_b):

    par = np.polyfit(np.log10(k[(k>low_b) & (k<up_b)]), np.log10(Y[(k>low_b) & (k<up_b)]), 1)
    exp = par[0]
    amp = par[1]
    print(exp,amp)
    return exp, amp   

## execute program

In [ ]:
df_t_psd = pd.DataFrame(columns = name_list)
df_t_freq = pd.DataFrame(columns = name_list)

i=0
for file in fname_list[0:2]:
    print(name_list[i])
    y_2D = process_phase(file)
    y_1D = np.mean(y_2D,axis=(1,2))
    f, psd = welch(y_1D,
               fs=.001,  # sample rate
               window='blackman',   # apply a Hanning window before taking the DFT
               nperseg=1024,        # compute periodograms of 256-long segments of x
               detrend='constant') # detrend x by subtracting the mean
    df_t_psd[name_list[i]] = pd.Series(f)
    df_t_freq[name_list[i]] = pd.Series(psd)
    i=i+1

## Save csv

In [ ]:
df_t_psd.to_csv(save_path+'t_psd'+'_'+dstr+'.txt')
df_t_freq.to_csv(save_path+'t_freq'+'_'+dstr+'.txt')

# Group the data

In [ ]:
df = pd.DataFrame({'fname_list':fname_list,'name_list':name_list})
df['dts'] = pd.to_datetime(df['name_list'],format='%Y.%m.%d_%H.%M.%S',utc=True)
df['delta_t'] = df['dts']-df['dts'].shift(1)
df.loc[0,'delta_t'] = pd.Timedelta(0)

def bin_f(row):
    if row['delta_t'] > pd.Timedelta('20H'):
        return row['dts'].date()
    
df['bin']=df.apply(bin_f,axis = 1)
df.loc[0:1,'bin']=df.loc[0,'dts'].date()
df['bin']=df['bin'].fillna(method='ffill')
g_df = df.groupby('bin')

Create a dataframe with the average psd measured in each night window. Column names contain date at the start of the night 

In [ ]:
df_avg_t_psd = pd.DataFrame()

for name,group in g_df:
    df_avg_t_psd.loc[:,str(name)] = df_t_psd.loc[:,group['name_list'].tolist()].mean(axis=1)

In [ ]:
df_avg_t_freq = pd.DataFrame()

for name,group in g_df:
    df_avg_t_freq.loc[:,str(name)] = df_t_freq.loc[:,group['name_list'].tolist()].mean(axis=1)

Save grouped data

In [ ]:
df_avg_t_psd.to_csv(save_path+'avg_t_psd_'+dstr+'.txt')
df_avg_t_freq.to_csv(save_path+'avg_t_freq_'+dstr+'.txt')